# Lesson 3: Embedding Models for Retrieval

**Objective**: Understand the role of embeddings in representing document chunks for retrieval.

**Topics**:
- Overview of embedding models: LLM-Embedder, BAAI/bge, etc.
- Selecting the right embedding model
- Integrating embeddings into the retrieval pipeline

**Practical Task**: Implement and test embedding models on the chunked documents.

**Resources**:
- Choosing and embedding model
- How to select an embedding model
- [Mastering RAG: How to Select an Embedding Model](https://www.rungalileo.io/blog/mastering-rag-how-to-select-an-embedding-model#:~:text=Embeddings%20encode%20the%20semantics%20of,efficient%20and%20user%20friendly%20experience.)
- [Vector Embeddings in RAG Applications](https://wandb.ai/mostafaibrahim17/ml-articles/reports/Vector-Embeddings-in-RAG-Applications--Vmlldzo3OTk1NDA5)
- [Vector Embeddings in RAG Applications](https://medium.com/thedeephub/vector-embeddings-in-rag-applications-9ea8043c172b)
- [Embeddings leaderboard](https://huggingface.co/spaces/mteb/leaderboard)


## Load the datasets

In [1]:
from langchain_community.document_loaders import PDFMinerLoader

file_path = "../data/Regulaciones cacao y chocolate 2003.pdf"
loader = PDFMinerLoader(file_path)
splitted_doc = loader.load_and_split()

In [ ]:
splitted_doc

## Dense Embeddings

-Dense embeddings are continuous, low-dimensional vectors where each dimension holds meaningful information. They are typically generated using neural networks and represent data in a compressed form. Dense embeddings are commonly used in deep learning models, capturing semantic similarity between inputs like documents and queries. In RAG, dense embeddings help retrieve documents that are semantically similar to a query.

In [ ]:
from dotenv import load_dotenv
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import RetrievalMode
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

embeddings = OpenAIEmbeddings()
open_source_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")


qdrant = QdrantVectorStore.from_documents(
    splitted_doc,
    embedding=open_source_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

query = "What did the president say about Ketanji Brown Jackson"
found_docs = qdrant.similarity_search(query)

In [ ]:
found_docs

### Huggingface embeddings

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

In [ ]:
len(query_result)

## Sparse embeddings

- Sparse embeddings are high-dimensional vectors, where most elements are zero, and only a few dimensions hold non-zero values. They are often derived from traditional methods like TF-IDF or BM25, focusing on specific keywords or tokens present in the text. In the context of RAG, sparse embeddings help match documents and queries based on exact term overlap rather than broader semantic relationships.

In [ ]:
from fastembed.sparse.bm25 import Bm25

bm25_embedding_model = Bm25("Qdrant/bm25")
bm25_embeddings = list(bm25_embedding_model.passage_embed(splitted_doc[0].page_content))
len(bm25_embeddings[0].values)

In [ ]:
sparse_embeddings_model = FastEmbedSparse(model_name="Qdrant/bm25")
sparse_embeddings = sparse_embeddings_model.embed_query("This is a test document.")
sparse_embeddings

## Late interaction embeddings

- Late interaction embeddings refer to a hybrid approach where sparse and dense embeddings are combined during the retrieval process, but their interactions are delayed until the final ranking or matching phase. This allows the model to first retrieve candidates based on simpler, faster methods and then refine the ranking through more complex, dense representations. In RAG, this helps improve retrieval quality while balancing computational efficiency.

In [ ]:
from fastembed.late_interaction import LateInteractionTextEmbedding

late_interaction_embedding_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")
late_interaction_embeddings = list(late_interaction_embedding_model.passage_embed(splitted_doc[0].page_content))
len(late_interaction_embeddings[0])